<a href="https://colab.research.google.com/github/hellog2n/Example-Tensorflow-Python/blob/main/VGG_model_%EB%B3%B5%EC%82%AC_%EB%B0%8F_%EB%A6%AC%EC%82%AC%EC%9D%B4%EC%A6%88_%ED%85%8C%EC%8A%A4%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VGG Net 모델로 이미지 예측하기

* VGG의 사이즈는 224*224로 고정하였습니다.
* 이미지를 갖고와서 리사이즈 하는 방식에 따라 다른 결과값이 도출되는 것 같습니다.
* VGG 모델의 레이어를 복사해서 사용하는 방식은 기존의 VGG 모델과 같은 결과를 출력하기에 문제가 없는 것으로 판단됩니다.

### 라이브러리 갖고오기
### VGG 모델 임포팅

In [1]:
from skimage.transform import resize
from numpy import asarray
from tensorflow.python.keras.applications.vgg16 import VGG16
import tensorflow as tf
import numpy as np
import cv2
from tensorflow.keras.applications.vgg16 import decode_predictions
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input

# VGG Model
size = 224

# Default Input Size for VGG : 224*224
vgg_model = VGG16()


553467904/553467096 [==============================] - 4s 0us/step


### Method1
기존에 사용하고 있던 리사이즈 방식 

- new_shape =  (224, 224, 3)
- cv2 함수를 이용하여 이미지 로드
- 이미지를 대량으로 갖고와서 skimage의 Resize 함수를 이용하여 사이즈를 바꾼다.
- 이미지 Preprocessing 이후 모델에 입력으로 주어진다. (Method2와 같음)

In [2]:
# -------------------------- Method 1 --------------------------------
# Method1 이미지 갖고오기
def load_Image(path):
  img = cv2.cvtColor(cv2.imread(path, cv2.IMREAD_COLOR), cv2.COLOR_BGR2RGB)
  return img

# Method1 이미지 크기 조정
def scale_images_GPU(images, new_shape):
    images_list = list()
    for image in images:
        new_image = resize(image, new_shape, 0)
        images_list.append(new_image)
    return asarray(images_list)

# Method1 이미지 리사이즈 후 모델에 입력
def Get_Class_Method1(inputs, model):
    """Compose the preprocess"""
    """ 이미지 리사이즈 224"""
    # GPU 설정
    with tf.device('/job:localhost/replica:0/task:0/device:GPU:0'):
      inputs = scale_images_GPU(inputs, (224,224,3))
      inputs = tf.keras.applications.vgg16.preprocess_input(inputs)
      return model.predict(inputs)

### Method2
출처 : https://machinelearningmastery.com/use-pre-trained-vgg-model-classify-objects-photographs/ 

img = load_img(imageName"String", target_size=(224, 224))
- keras.preprocessing.image의 함수인 load_img 함수를 이용하여 사이즈를 지정하여 로드한다.
- 이후 이미지를 array로 변환하고 Reshape 한다.
- 이미지 Preprocessing 이후 모델에 입력으로 주어진다. (Method1와 같음)


In [3]:
# -------------------------- Method 2 --------------------------------

# 이미지 리사이즈 후 모델에 입력
def Get_Class_Method2(image, model):
  with tf.device('/job:localhost/replica:0/task:0/device:GPU:0'):
    # convert the image pixels to a numpy array
    image = img_to_array(image)
    # reshape data for the model
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    # prepare the image for the VGG model
    image = preprocess_input(image)
    # predict the probability across all output classes
    yhat = model.predict(image)
  return yhat

### VGG 모델 사용하기
VGG 모델의 클래스 예측을 확인하기 위해 모델에서 레이어를 제거하지 않고 그대로 사용하였다. 


Method 방식에 따라 이미지 로드 및 리사이즈 할 방식을 다룬다.

```
embed_image_in_VGG16(이미지, 사용할 Method 방식(integer))
```
* VGG 모델을 복사하여 사용한다. model을 임시로 선언하고, 이 모델에 VGG 모델의 모든 레이어를 복사하여 추가한다. 복사한 모델의 경우, 모델이 예측시 Train되지 않도록 False값을 설정하였다.

```
embed_image_in_VGG162(이미지, 사용할 Method 방식(integer))
```
* VGG 모델을 그대로 사용한다.





In [4]:
# 모델을 복사하여 사용한다.
def embed_image_in_VGG16(img, method):
    # 모델 복사
    model = tf.keras.Sequential()
    for layer in vgg_model.layers[:]:
        model.add(layer)

    # 마지막 레이어 제거 (Classess 1000)
    #model.layers.pop()

    # 모든 레이어가 학습되지 않도록 적용
    for layer in model.layers:
        layer.trainable = False
    if method == 1:
      Values = Get_Class_Method1(img, model=model)
    elif method == 2:
      Values = Get_Class_Method2(img, model=model)
    else:
      raise NotImplementedError
    return Values


# VGG 모델 원본 그대로를 사용한다.
def embed_image_in_VGG162(img, method):
    if method == 1:
      Values = Get_Class_Method1(img, model=vgg_model)
    elif method == 2:
      Values = Get_Class_Method2(img, model=vgg_model)
    else:
      raise NotImplementedError
    return Values



### Main Method

`imageName = "이미지_이름.확장자" 에 모델의 입력으로 주어질 이미지의 이름과 확장자를 선언한다.`

### 고양이 이미지들로 테스트를 해보았습니다.


###예시 이미지 1
![cat3.jpeg](https://drive.google.com/uc?export=view&id=14zn7Gt_Q8X0c5SPv-wjjA549mVPOWkjP)

결과값 :

Method1(VGG 복사, VGG 원본) ->
matchstick (7.70%)
matchstick (7.70%)

Method2(VGG 복사, VGG 원본) ->
tiger_cat (15.25%)
tiger_cat (15.25%)



---


### 예시 이미지 2

![cat2.png](https://drive.google.com/uc?export=view&id=1bN9w3urpjDuy0ji_ZheqyDAUS9JkURq7)

결과값 :

Method1(VGG 복사, VGG 원본) ->
matchstick (7.61%)
matchstick (7.61%)

Method2(VGG 복사, VGG 원본) ->
carton (14.74%)
carton (14.74%)

---

### 예시 이미지 3

![cat.jpeg](https://drive.google.com/uc?export=view&id=19uLhViuBw3nOpr5Gsgr8D1SxwFjLj1Ny)

결과값 :

Method1(VGG 복사, VGG 원본) ->
matchstick (7.55%)
matchstick (7.55%)


Method2(VGG 복사, VGG 원본) ->
Egyptian_cat (56.56%)
Egyptian_cat (56.56%)




In [6]:
# cat3.jpeg

imageName = "이미지_이름.확장자"

def main(imageName):
  Method1(imageName)
  Method2(imageName)
  

def Method1(imageName):
  # Method 1

  path = imageName
  img = load_Image(path)
  
  # VGG 복사된 모델 => 함수 매개변수 (이미지, Method1 선택)
  yhat = embed_image_in_VGG16(img, 1)
  label = decode_predictions(yhat)
  label = label[0][0]
  print("%s (%.2f%%)" % (label[1], label[2] * 100))

  # VGG 원본 모델 => 함수 매개변수 (이미지, Method1 선택)
  yhat2 = embed_image_in_VGG162(img, 1)
  label2 = decode_predictions(yhat2)
  label2 = label2[0][0]
  print("%s (%.2f%%)" % (label2[1], label2[2] * 100))


def Method2(imageName):
  # Method 2  
  img = load_img(imageName, target_size=(224, 224))
  # VGG 복사된 모델 => 함수 매개변수 (이미지, Method2 선택)
  yhat = embed_image_in_VGG16(img, 2)
  label = decode_predictions(yhat)
  label = label[0][0]
  print("%s (%.2f%%)" % (label[1], label[2] * 100))

  # VGG 원본 모델 => 함수 매개변수 (이미지, Method2 선택)
  yhat2 = embed_image_in_VGG162(img, 2)
  label2 = decode_predictions(yhat2)
  label2 = label2[0][0]
  print("%s (%.2f%%)" % (label2[1], label2[2] * 100))


main()

40960/35363 [==================================] - 0s 0us/step
matchstick (7.70%)
matchstick (7.70%)
tiger_cat (15.25%)
tiger_cat (15.25%)
